In [1]:
import os
import os.path

import numpy as np
import tensorflow as tf

import test_cifar
import architecture
import tools



conv1_1
weights shape:  (3, 3, 3, 64)
biases shape:  (64,)


conv1_2
weights shape:  (3, 3, 64, 64)
biases shape:  (64,)


conv2_1
weights shape:  (3, 3, 64, 128)
biases shape:  (128,)


conv2_2
weights shape:  (3, 3, 128, 128)
biases shape:  (128,)


conv3_1
weights shape:  (3, 3, 128, 256)
biases shape:  (256,)


conv3_2
weights shape:  (3, 3, 256, 256)
biases shape:  (256,)


conv3_3
weights shape:  (3, 3, 256, 256)
biases shape:  (256,)


conv4_1
weights shape:  (3, 3, 256, 512)
biases shape:  (512,)


conv4_2
weights shape:  (3, 3, 512, 512)
biases shape:  (512,)


conv4_3
weights shape:  (3, 3, 512, 512)
biases shape:  (512,)


conv5_1
weights shape:  (3, 3, 512, 512)
biases shape:  (512,)


conv5_2
weights shape:  (3, 3, 512, 512)
biases shape:  (512,)


conv5_3
weights shape:  (3, 3, 512, 512)
biases shape:  (512,)


fc6
weights shape:  (25088, 4096)
biases shape:  (4096,)


fc7
weights shape:  (4096, 4096)
biases shape:  (4096,)


fc8
weights shape:  (4096, 1000)
biases shap

In [2]:
IMG_W = 32
IMG_H = 32
N_CLASSES = 10
BATCH_SIZE = 24
learning_rate = 0.01
MAX_STEP = 15001   
IS_PRETRAIN = True

In [ ]:
def train():
    
    pre_trained_weights = '../vgg16.npy'
    data_dir = '../../data/cifar-10-batches-bin/'
    train_log_dir = 'logs/train/'
    val_log_dir = 'logs/val/'
    
    with tf.name_scope('input'):
        tra_image_batch, tra_label_batch = test_cifar.read_cifar10(data_dir=data_dir,
                                                 is_train=True,
                                                 batch_size= BATCH_SIZE,
                                                 shuffle=True)
        val_image_batch, val_label_batch = test_cifar.read_cifar10(data_dir=data_dir,
                                                 is_train=False,
                                                 batch_size= BATCH_SIZE,
                                                 shuffle=False)
        
    x = tf.placeholder(tf.float32, shape=[BATCH_SIZE, IMG_W, IMG_H, 3])
    y_ = tf.placeholder(tf.int16, shape=[BATCH_SIZE, N_CLASSES]) 
    
    logits = architecture.VGG16N(x, N_CLASSES, IS_PRETRAIN)
    loss = tools.loss(logits, y_)
    accuracy = tools.accuracy(logits, y_)
    
    my_global_step = tf.Variable(0, name='global_step', trainable=False) 
    train_op = tools.optimize(loss, learning_rate, my_global_step)   
    
    saver = tf.train.Saver(tf.global_variables())
    summary_op = tf.summary.merge_all()   
       
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    
    # load the parameter file, assign the parameters, skip the specific layers
    tools.load_with_skip(pre_trained_weights, sess, ['fc6','fc7','fc8'])   


    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)    
    tra_summary_writer = tf.summary.FileWriter(train_log_dir, sess.graph)
    val_summary_writer = tf.summary.FileWriter(val_log_dir, sess.graph)
    
    try:
        for step in np.arange(MAX_STEP):
            if coord.should_stop():
                    break
                
            tra_images,tra_labels = sess.run([tra_image_batch, tra_label_batch])
            _, tra_loss, tra_acc = sess.run([train_op, loss, accuracy],
                                            feed_dict={x:tra_images, y_:tra_labels})            
            if step % 50 == 0:                 
                print ('Step: %d, loss: %.4f, accuracy: %.4f%%' % (step, tra_loss, tra_acc))
                summary_str = sess.run(summary_op,
                                      feed_dict={x:tra_images, y_:tra_labels})
                tra_summary_writer.add_summary(summary_str, step)
                
            if step % 200 == 0:
                val_images, val_labels = sess.run([val_image_batch, val_label_batch])
                val_loss, val_acc = sess.run([loss, accuracy],
                                             feed_dict={x:val_images,y_:val_labels})
                print('**  Step %d, val loss = %.2f, val accuracy = %.2f%%  **' %(step, val_loss, val_acc))

                summary_str = sess.run(summary_op,
                                      feed_dict={x:val_images, y_:val_labels})
                val_summary_writer.add_summary(summary_str, step)
                    
            if step % 2000 == 0:
                checkpoint_path = os.path.join(train_log_dir, 'model.ckpt')
                saver.save(sess, checkpoint_path, global_step=step)
                
    except tf.errors.OutOfRangeError:
        print('Done training -- epoch limit reached')
    finally:
        coord.request_stop()
        
    coord.join(threads)
    sess.close()


In [ ]:
train()

Step: 0, loss: 3.0521, accuracy: 0.0000%
**  Step 0, val loss = 2.50, val accuracy = 8.33%  **
Step: 50, loss: 2.2929, accuracy: 12.5000%
Step: 100, loss: 2.6431, accuracy: 0.0000%
Step: 150, loss: 2.3864, accuracy: 12.5000%
Step: 200, loss: 2.3982, accuracy: 12.5000%
**  Step 200, val loss = 2.57, val accuracy = 4.17%  **
Step: 250, loss: 2.4337, accuracy: 16.6667%
Step: 300, loss: 2.3135, accuracy: 25.0000%
Step: 350, loss: 2.3513, accuracy: 12.5000%
Step: 400, loss: 2.3043, accuracy: 12.5000%
**  Step 400, val loss = 2.43, val accuracy = 16.67%  **
Step: 450, loss: 2.4233, accuracy: 12.5000%
Step: 500, loss: 2.2049, accuracy: 8.3333%
Step: 550, loss: 2.2559, accuracy: 12.5000%
Step: 600, loss: 2.2676, accuracy: 25.0000%
**  Step 600, val loss = 2.36, val accuracy = 16.67%  **
Step: 650, loss: 2.3372, accuracy: 8.3333%
Step: 700, loss: 2.5736, accuracy: 8.3333%
Step: 750, loss: 2.4977, accuracy: 16.6667%
Step: 800, loss: 2.3906, accuracy: 12.5000%
**  Step 800, val loss = 2.33, val a

Step: 6950, loss: 2.1834, accuracy: 12.5000%
Step: 7000, loss: 2.4980, accuracy: 29.1667%
**  Step 7000, val loss = 2.16, val accuracy = 41.67%  **
Step: 7050, loss: 2.2742, accuracy: 4.1667%
Step: 7100, loss: 2.5744, accuracy: 8.3333%
Step: 7150, loss: 2.3694, accuracy: 12.5000%
Step: 7200, loss: 2.9421, accuracy: 0.0000%
**  Step 7200, val loss = 2.32, val accuracy = 8.33%  **
